In [1]:
import pip


In [9]:
!pip install bs4

     |████████████████████████████████| 112kB 7.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [11]:
!pip install geopy


     |████████████████████████████████| 112kB 6.7MB/s eta 0:00:01


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")


Libraries imported.


In [21]:
import requests

In [23]:

data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [24]:
headers = []
rows = []
first_row = True
for group in soup.table.find_all('tr'):
    # Using split()[0] because last item of each row had \n in it
    if first_row:
        headers = [header.text.split('\n')[0] for header in group.find_all('th')]
        first_row = False
    else:
        row = [value.text.split('\n')[0] for value in group.find_all('td')]
        rows.append(row)
print('Headers: ',headers)
print('First 5 rows: ',rows[:6])

Headers:  ['Postal code', 'Borough', 'Neighborhood']
First 5 rows:  [['M1A', 'Not assigned', ''], ['M2A', 'Not assigned', ''], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Regent Park / Harbourfront'], ['M6A', 'North York', 'Lawrence Manor / Lawrence Heights']]


In [25]:
df = pd.DataFrame(data=rows, columns=headers)
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [26]:

df.shape

(180, 3)

In [27]:
df.replace('Not assigned', np.NaN, inplace=True)

# Select only not-null values of 'Borough'
df = df[pd.notnull(df['Borough'])]
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [28]:

print('New shape: ', df.shape)
print('Any NaN value in Borough: ', df['Borough'].isnull().any())

New shape:  (103, 3)
Any NaN value in Borough:  False


In [30]:
df[pd.isnull(df['Neighborhood'])]

,Postal code,Borough,Neighborhood


In [32]:
df[pd.isnull(df['Neighborhood'])]

,Postal code,Borough,Neighborhood


In [34]:
df = df.groupby(['Postal code','Borough']).Neighborhood.apply(lambda x: ', '.join(x)).to_frame()
df.reset_index(inplace=True)
df.head(10)

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


In [35]:

print('Grouped DF shape: ', df.shape)

Grouped DF shape:  (103, 3)
